# Modelos lineares e regularização

Vamos começar importando as bibliotecas de costume

In [1]:
%matplotlib inline
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

## O que acontece se duas colunas forem iguais?

In [14]:
X = np.array(
    [
        [1,1,7,7],
        [1,2,8,3],
        [1,3,7,3],
        [1,4,6,3],
        [1,5,5,3],
        [1,6,4,3],
        [1,7,3,3],
        [1,8,2,3]

    ]
)
X

array([[1, 1, 7, 7],
       [1, 2, 8, 3],
       [1, 3, 7, 3],
       [1, 4, 6, 3],
       [1, 5, 5, 3],
       [1, 6, 4, 3],
       [1, 7, 3, 3],
       [1, 8, 2, 3]])

In [15]:
XtX= X.T @ X
XtX

array([[  8,  36,  42,  28],
       [ 36, 204, 154, 112],
       [ 42, 154, 252, 154],
       [ 28, 112, 154, 112]])

In [16]:
np.linalg.det(XtX)

0.0

In [17]:
np.linalg.inv(XtX)

LinAlgError: Singular matrix

Vamos tambem inicializar o gerador de números aleatórios do Numpy para garantir a reproducibilidade dos nossos resultados.

In [ ]:
RAND_SEED = 42
np.random.seed(RAND_SEED)


Agora vamos gerar um conjunto de valores experimentais $(x,y)$ aleatórios da seguinte forma:

- Os valores de $x$ são sorteados uniformemente entre $-3$ e $3$
- Os valores de $y$ são obtidos em dois passos:
    - Calculamos $y_{\text{clean}}$ como função de $x$: $y_{\text{clean}} = \frac{1}{2} x^2 + x + 2$
    - Calculamos o valor final de $y$ como sendo $y = y_{\text{clean}} + \varepsilon$ onde $\varepsilon \sim N(0, 1)$ é uma variável aleatória normal de média zero e desvio padrão $1$.

Vamos usar esses valores experimentais para continuar nossa exploração sobre modelos lineares.

In [ ]:
# Numero de pontos a serem gerados.
m = 50

# Valores de x, na forma de uma matriz-coluna m por 1.
X = 6 * np.random.rand(m, 1) - 3

# Valores de y sem ruido, que são uma função quadrática de x.
y_clean = 0.5 * X**2 + X + 2

# Adicionando ruido à y.
y = y_clean + np.random.randn(m, 1)

# Vamos ver como ficou.
plt.figure(figsize=(8, 6))
plt.plot(X, y, "b.", linewidth=3)
plt.show()


Nosso objetivo será ajustar vários modelos lineares polinomiais aos dados, com diferentes graus de polinômio, e compará-los:

- Modelo linear de grau $1$: $\hat{y} = \theta_0 + \theta_1 x$
- Modelo linear de grau $2$: $\hat{y} = \theta_0 + \theta_1 x + \theta_2 x^2$
- Modelo linear de grau $30$: $\hat{y} = \theta_0 + \theta_1 x + \cdots + \theta_{30} x^{30}$

In [ ]:
def experimento(X, y, degree):
    # Cria a pipeline.
    pipe = Pipeline([
        ("poly_features",
         PolynomialFeatures(
             degree=degree,
             include_bias=False,
         )),
        ("std_scaler", StandardScaler()),
        ("lin_reg", LinearRegression()),
    ])

    # Ajusta a pipeline nos dados de treinamento.
    pipe.fit(X, y)

    # Imprime os parametros obtidos no ajuste de modelo.
    def print_title(title):
        msg = f'Parâmetros do {title}'
        print(msg)
        print('-' * len(msg))

    def print_array(name, arr):
        print(name)
        pprint(arr)
        print()

    scaler = pipe.named_steps['std_scaler']
    print_title('StandardScaler')
    print_array('mean:', scaler.mean_)
    print_array('scale:', scaler.scale_)

    model = pipe.named_steps['lin_reg']
    print_title('modelo linear')
    print_array('theta_0:', model.intercept_)
    print_array('Demais coeficientes theta_i:', model.coef_)

    # Coordenadas de teste para visualizar as predições efetuadas
    # pelo modelo ajustado.
    X_test = np.linspace(-3, 3, num=100).T

    # Calcula as predições da pipeline nos dados de teste.
    y_test = pipe.predict(X_test.reshape(-1, 1))

    # Vamos ver como ficou.
    plt.figure(figsize=(8, 6))
    plt.plot(X, y, "b.", linewidth=3)
    plt.plot(X_test, y_test, 'r-', label=f'grau {degree}')
    plt.xlabel('$X$')
    plt.ylabel('$y$', rotation=0)
    plt.axis([-3, 3, 0, 10])
    plt.title(f'Modelo de grau ${degree}$')
    plt.show()


No primeiro experimento, vamos usar um modelo linear de grau $1$:

In [ ]:
# Experimento 1: grau baixo.
experimento(X, y, degree=1)


No segundo experimento vamos usar um modelo polinomial de grau $2$. Este é o modelo que esperamos que atinja nosso *benchmark* de parâmetros.

In [ ]:
# Experimento 2: grau adequado.
experimento(X, y, degree=2)


No terceiro modelo vamos exagerar no grau do modelo polinomial, para ver o que acontece.

In [ ]:
# Experimento 3: grau exagerado.
experimento(X, y, degree=30)


# Regularização

Conforme vimos acima, um modelo muito complexo pode sofrer do problema de *overfitting*. Uma forma de domar a complexidade do modelo é limitar o número de *features*. Por exemplo, usando *features* polinomiais, podemos limitar o grau do polinômio.

Outra forma de tratar a complexidade dos modelos é através da estratégia de *regularização*. Trata-se do seguinte:

1. Definimos métricas diferentes de erro para a fase de treinamento e a fase de testes.
    - Na fase de treinamento usamos uma *métrica regularizada*
    - Na fase de testes usamos uma métrica convencional
    
2. A métrica regularizada funciona assim: trata-se da métrica convencional acrescida de um termo que *penaliza a complexidade do modelo*.

Com isso favorecemos modelos de complexidade reduzida, mesmo que o grau do polinômio seja alto.

## Ridge, Lasso e ElasticNet

Nestas modalidades de regularização adicionamos uma penalidade para a norma do vetor de parâmetros:

- Ridge: a penalidade é proporcional à norma $L_2$ do vetor de parâmetros.

- Lasso: a penalidade é proporcional à norma $L_1$ do vetor de parâmetros.

- ElasticNet: uma soma ponderada de penalidades proporcionais às normas $L_1$ e $L_2$ do vetor de parâmetros é aplicada.


**Atividade:** O material do livro-texto, na seção "Regularized Linear Models" está muito bom, estude este material e responda:

- Qual a diferença entre *ridge regression*, *lasso regression*, e *elastic net*?

**R:**

Eis aqui uma demonstração das várias regularizações:

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


def outro_experimento(msg, pipeline, X_train, y_train, X_test, y_test):
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    RMSE = np.sqrt(mean_squared_error(y_pred, y_test))

    model = pipeline.named_steps['lin_reg']
    print(f'{msg}:')
    print(f'intercept = {model.intercept_}')
    print(f'coefs = {model.coef_}')
    print(f'RMSE: {RMSE}')
    print()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RAND_SEED,
)


In [ ]:
def get_linear_regressor_pipeline(degree):
    pipe = Pipeline([
        ("poly_features",
         PolynomialFeatures(
             degree=degree,
             include_bias=False,
         )),
        ("std_scaler", StandardScaler()),
        ("lin_reg", LinearRegression()),
    ])
    return pipe


poly_reg_1 = get_linear_regressor_pipeline(degree=1)
poly_reg_2 = get_linear_regressor_pipeline(degree=2)
poly_reg_30 = get_linear_regressor_pipeline(degree=30)


In [ ]:
# Testa o fit do poly_reg_1.
outro_experimento(
    'Fit de grau 1, sem regularização',
    poly_reg_1,
    X_train,
    y_train,
    X_test,
    y_test,
)

# Testa o fit do poly_reg_2.
outro_experimento(
    'Fit de grau 2, sem regularização',
    poly_reg_2,
    X_train,
    y_train,
    X_test,
    y_test,
)

# Testa o fit do poly_reg_30.
outro_experimento(
    'Fit de grau 30, sem regularização',
    poly_reg_30,
    X_train,
    y_train,
    X_test,
    y_test,
)


In [ ]:
# Coeficiente de regularização para os experimentos a seguir.
alpha = 1e-1


In [ ]:
# Testa o fit da regularização ridge.
poly_reg_ridge = Pipeline([
    ("poly_features", PolynomialFeatures(degree=30, include_bias=False)),
    ("std_scaler", StandardScaler()),
    ("lin_reg", Ridge(alpha=alpha)),
])
outro_experimento(
    'Fit de grau 30, regularização ridge',
    poly_reg_ridge,
    X_train,
    y_train,
    X_test,
    y_test,
)


In [ ]:
# Test o fit da regularização lasso.
poly_reg_lasso = Pipeline([
    ("poly_features", PolynomialFeatures(degree=30, include_bias=False)),
    ("std_scaler", StandardScaler()),
    ("lin_reg", Lasso(alpha=alpha)),
])
outro_experimento(
    'Fit de grau 30, regularização lasso',
    poly_reg_lasso,
    X_train,
    y_train,
    X_test,
    y_test,
)


In [ ]:
# Test o fit da regularização elastic net.
poly_reg_elasticnet = Pipeline([
    ("poly_features", PolynomialFeatures(degree=30, include_bias=False)),
    ("std_scaler", StandardScaler()),
    ("lin_reg", ElasticNet(alpha=alpha, l1_ratio=0.5, random_state=RAND_SEED)),
])
outro_experimento(
    'Fit de grau 30, regularização elastic net',
    poly_reg_elasticnet,
    X_train,
    y_train,
    X_test,
    y_test,
)


**Atividade:** 

- Coloque o $\alpha$ bem alto, o que acontece?

- Coloque o $\alpha$ muito baixo, o que acontece?

- Explique a diferença observada entre sem regularização / regularização ridge / regularização lasso / regularização elastic net.

**R:**

## Regularização por parada prematura (*early stopping*)

Um modo bizarro de regularização é o chamado *early stopping*.

Considere o algoritmo de treinamento *gradient descent*. Quanto mais iteramos neste algoritmo (em machine learning, as iterações são chamadas **epochs**), menor o *erro de treinamento*. Porém, se acompanharmos o *erro de validação* à cada epoch, vemos que este decresce com as epochs até um certo ponto, *e depois começa a subir novamente*!

<img src="early_stopping_plot.png" alt="Regularização por parada prematura" style="width: 600px;"/>


**Atividade:** Por que isso acontece?

**R:**

Continuando:

Portanto, se detectarmos que o erro de validação está realmente subindo, podemos parar com o processo de treinamento e adotar o modelo resultante como o nosso modelo treinado! Esta estratégia de *parada prematura* (early stopping) é surpreendentemente simples e eficaz!

# Interpretabilidade

In [ ]:
import statsmodels.api as sm


In [ ]:
pipe = PolynomialFeatures(degree=5, include_bias=True)
Xb = pipe.fit_transform(X)

model = sm.OLS(y, Xb)
results = model.fit()


In [ ]:
results.summary()


**Atividade:**

Experimente com diferentes valores para o grau do polinômio: 1, 2, 5, 10. Relate o que você observou

**R:**